# Setup Environment

You will need the normalizing_flows.zip in the /content folder

In [1]:
from IPython.display import clear_output

In [2]:
!pip install mido
!pip install git+https://github.com/bgroenks96/normalizing-flows
!pip install mlflow
!pip install pyarrow
!pip install tensorflow_io
clear_output(wait=False)


In [3]:
import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow import feather
import mido
from mido import Message, MetaMessage
from mido import MidiFile, MidiTrack
import copy
import re, os
import os.path
import tensorflow_io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV

In [4]:
# -o so we don't have to interact with this after execution.
!unzip -o /content/normalizing_flows.zip -d /usr/local/lib/python3.7/dist-packages/
clear_output(wait=False)

In [5]:
import mlflow
import tensorflow as tf
import normalizing_flows as nf
from normalizing_flows.models.vae import GatedConvVAE

from normalizing_flows.flows import Flow
from normalizing_flows.flows.affine import Planar
from normalizing_flows.flows.sylvester import TriangularSylvester
from urllib.request import urlretrieve

tf.compat.v1.enable_eager_execution(device_policy = "silent")
tf.config.run_functions_eagerly(False)
tf.data.experimental.enable_debug_mode()

import torch
PADDING = 2000

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adamax.py:90: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Clean the weird shit that happened in the midis

In [ ]:
#!unzip -o midis_2.zip -d .
#directory = '/content/midis_2/'
#for filename in os.listdir(directory):
#    f = os.path.join(directory, filename)
#    if not filename.endswith(".midi"):
#        newname = filename[0:4] + ".midi"
#        n = os.path.join(directory, newname)
#    else:
#        newname = filename[0:4] + ".midi"
#        n = os.path.join(directory, newname)
#    # checking if it is a file
#    os.rename(f, n)
#clear_output(wait=False)


### delete the broken ones

In [ ]:
dead_files = ['3767.midi','4211.midi','3395.midi','3396.midi','4930.midi','3403.midi','3420.midi','3768.midi','3384.midi','3389.midi','4555.midi','4192.midi','4367.midi','4037.midi','4786.midi','1414.midi','3423.midi','4198.midi','3416.midi','3426.midi','3735.midi','3404.midi','3771.midi','4278.midi','3388.midi','4556.midi','3038.midi','4863.midi','4915.midi','3385.midi','4929.midi','3380.midi','3419.midi','2100.midi','4213.midi','3776.midi','3422.midi','4309.midi','3397.midi','4200.midi','4368.midi','3409.midi','4770.midi','4217.midi','3918.midi','4781.midi','4199.midi','3425.midi','3394.midi','3407.midi','3410.midi','4215.midi','3772.midi','4360.midi','3528.midi','2378.midi','4194.midi','3401.midi','4212.midi','3775.midi','3402.midi','2685.midi','1870.midi','3770.midi','4777.midi','3769.midi','3774.midi','3399.midi','4864.midi','4191.midi','4338.midi','4197.midi','2782.midi','4195.midi','3382.midi','3414.midi','2187.midi','2214.midi','3406.midi','3411.midi','4218.midi','1767.midi','3424.midi','3417.midi','4303.midi','4193.midi','4196.midi','3773.midi','4862.midi']
#for f in dead_files:
#    os.remove('midis_2/' + f)
dead_files.sort()
print(dead_files)

['1414.midi', '1767.midi', '1870.midi', '2100.midi', '2187.midi', '2214.midi', '2378.midi', '2685.midi', '2782.midi', '3038.midi', '3380.midi', '3382.midi', '3384.midi', '3385.midi', '3388.midi', '3389.midi', '3394.midi', '3395.midi', '3396.midi', '3397.midi', '3399.midi', '3401.midi', '3402.midi', '3403.midi', '3404.midi', '3406.midi', '3407.midi', '3409.midi', '3410.midi', '3411.midi', '3414.midi', '3416.midi', '3417.midi', '3419.midi', '3420.midi', '3422.midi', '3423.midi', '3424.midi', '3425.midi', '3426.midi', '3528.midi', '3735.midi', '3767.midi', '3768.midi', '3769.midi', '3770.midi', '3771.midi', '3772.midi', '3773.midi', '3774.midi', '3775.midi', '3776.midi', '3918.midi', '4037.midi', '4191.midi', '4192.midi', '4193.midi', '4194.midi', '4195.midi', '4196.midi', '4197.midi', '4198.midi', '4199.midi', '4200.midi', '4211.midi', '4212.midi', '4213.midi', '4215.midi', '4217.midi', '4218.midi', '4278.midi', '4303.midi', '4309.midi', '4338.midi', '4360.midi', '4367.midi', '4368.midi'

# The Code
Midi tensor class that reads both midi and npy files as tensors

In [6]:
# class for making tensors of midis
class MidiTensor():
    def __init__(self, fileName,padding=4000,clip=True):
        #print(fileName[-9:-5])
        self.padding = padding
        self.fileName = fileName
        if self.fileName.endswith('.midi') or self.fileName.endswith('.mid'):
            self.midi = MidiFile(fileName, clip=clip)
            self.preprocess()
            self.header = {}
            
            self.processHeader()
            #print(self.header)
            self.tracks =[]
            self.processSong()
            #for track in self.tracks: # debug
             #   for line in track:
              #      for item in line:
                        #if not type(item) == float:
                            #print(type(item), line)
            #    print(np.array(track).shape)
            self.tensor = tf.convert_to_tensor(self.tracks)
        # EXPERIMENTAL
        else:
            self.header = {}
            self.track = np.load(fileName, allow_pickle=True)
            #print(self.track.shape)
            self.track = self.track
            self.headerFromTensor()
            self.tensor = tf.convert_to_tensor(self.track)

    def removeMetaMessage(self,string):
        expression = r'MetaMessage\(\''
        out = re.sub(expression, '',string)
        out = out[:-1]
        return out
    def processHeader(self):
        sigs = {'Cb': 0, 'Gb': 1, 'Db': 2, 'Ab': 3, 'Eb': 4, 'Bb': 5, 
                'F': 6, 'C': 7, 'G': 8, 'D': 9, 'A': 10, 'E': 11, 'B': 12, 
                'F#': 13, 'C#': 14, 'Abm': 15, 'Ebm': 16, 'Bbm': 17, 'Fm': 18, 
                'Cm': 19, 'Gm': 20, 'Dm': 21, 'Am': 22, 'Em': 23, 'Bm': 24, 
                'F#m': 25, 'C#m': 26, 'G#m': 27, 'D#m': 28, 'A#m': 29, 'unk':30}
        head = self.midi.tracks[0]
        out = {'tempo':0,'key':0,'num':0,'den':0,'cpc':0,'nnpb':0}
        

        for msg in head:
            if   msg.type == 'set_tempo':
                out['tempo'] = (float(msg.tempo)/16777215)*255
            elif msg.type == 'key_signature':
                
                try:
                    out['key']   = float(sigs[msg.key])
                except KeyError: 
                    print(self.fileName)
                    out['key']   = 30.0
                    pass
            elif msg.type == 'time_signature':
                out['num']   = float(msg.numerator)
                out['den']   = float(msg.denominator)
                out['cpc']   = float(msg.clocks_per_click)
                out['nnpb']  = float(msg.notated_32nd_notes_per_beat)
        self.header = out
    def preprocess(self):
        message_numbers = []
        duplicates = []
        # Looping through the midi tracks
        for track in self.midi.tracks:
            # If there is a track length that matches a previously stored track length
            if len(track) in message_numbers:
                # Add to duplicate array
                duplicates.append(track)
            else:
                # if there is no match, add to message number array
                message_numbers.append(len(track))

        # Looping through duplicate array
        for track in duplicates:
            # Removing all recorded duplicate tracks from midi 
            self.midi.tracks.remove(track)
    def processSong(self):
        def changeAll(input, index, val):
            for i in range(len(input)):
                input[i][index] = val
            return input
        col_names =  ['message_type', 'channel', 'note', 'velocity', 'time', 'control',
                    'program', 'value', 'port', 'tempo', 'key', 'numerator',
                    'denominator', 'clocks_per_click', 'notated_32nd_notes_per_beat',
                    'pitch']
        col_ind   =  range(0,len(col_names))
        cols = dict(zip(col_names, col_ind))
        message_types = {'channel_prefix': 0, 'note_on': 1, 'sysex': 2, 'end_of_track': 3, 'control_change': 4,
                         'note_off': 5, 'program_change': 6, 'midi_port': 7,'pitchwheel':8, 'sequencer_specific':9}
        #print(cols)

        full_df = []
        # Looping through each midi track
        for j, track in enumerate(self.midi.tracks):
            if j == 0: continue
            co = pr = va = po = 0.0

            # Setting index and creating new dataframe
            index = range(0,len(track))
            lines = []
            
            pad = self.padding
            # Looping though the individual lines in each track
            
            normalizer = [10,15,127,127,1000,127,127,127,2,16777215,30,255,255,255,255,16384]
            for i in range(pad):
                temp = [2.161205992044448, 4.858070501475705, 54.2089394866137, 57.98296395518057, 
                        305.62208794690724, 28.438560581049853, 32.47665896335424, 49.10419280290525, 
                        0.07589963684384285, self.header['tempo'], self.header['key'], self.header['num'], 
                        self.header['den'], self.header['cpc'], self.header['nnpb'], (-242.74861137163506 + 8192)]

                #temp = [np.nan, np.nan, np.nan, np.nan, np.nan,
                 #       np.nan, np.nan, np.nan, np.nan, 
                  #      self.header['tempo'], self.header['key'], self.header['num'], self.header['den'], 
                   #     self.header['cpc'], self.header['nnpb'], np.nan]
                line = ''
                if i >= len(track): 
                    lines.append(temp)
                    continue

                line = track[i]

                split = []
                
                
                # If the line is a metamessage
                if type(line) == mido.midifiles.meta.MetaMessage:
                    # clean the line then assign each element of the line to split
                    line = self.removeMetaMessage(str(line))
                    line = line.replace("\'",'')
                    split = str(line).split(',')
                else:
                    # If not, no cleaning needed, assign each elemtn of the line to split
                    split = str(line).split()
                #print(split[0])
                message_type = split[0].strip()
                # If the first element in split is NOT one of the messages types, continue loop
                if not message_type in message_types.keys(): 
                    lines.append(temp)
                    continue
                # Getting number related to message type and storing into first element of dataframe
                #print(message_type)
                m_type = message_types[message_type]
                #print("m_type: ", m_type, "i: ", i)
                temp[cols['message_type']] = float(m_type)

                # Looping through the rest of the elements in split
                for entry in split[1:]:
                    # Getting only the values, removing un-needed characters
                    pattern = r'(.*)(=)(.*)'
                    capture = re.search(pattern,entry)

                    # If pattern was not found within the element, continue loop
                    if capture == None: continue

                    # Identifying the column name and value associated with it
                    column = capture.group(1).strip()
                    value_ = capture.group(3)
                    #Converting the value to the column variable type
                    if not column in col_names: continue
                    
                    if column == 'pitch': 
                        value = (float(value_)  + 8192)
                    elif column == 'key': value = float(value_)
                    else:
                        dig = r'(^[\d]*$)'
                        matchd = re.search(dig, value_)
                        if not matchd == None:
                            value = float(value_)
                        else:
                            value = value_
                    if column   == 'control':
                        co = value
                    elif column == 'program':
                        pr = value
                    elif column == 'value':
                        va = value
                    elif column == 'port':
                        po = value
                    #out[column] = value
                    # Assigning the index value and comlumn values to the dataframe
                    temp[cols[column]] = value
                out = temp # np.divide(temp,normalizer)
                #print(out)
                lines.append(out)
            # add padding
                
            # set the track header info
            #lines = changeAll(lines, cols['control'], co)
            #lines = changeAll(lines, cols['program'], pr)
            #lines = changeAll(lines, cols['value'], va)
            #lines = changeAll(lines, cols['port'], po)

            # set the song header info


            #print(my_df)
            # Append the dataframe to the array of dataframes
            #print(my_df.values.tolist())
            self.tracks.append(lines)

    def writeTensor(self,loc='./trackfiles/'):
        path = loc + self.fileName[-9:-5] + '_'

        # If the path does not exist, create directory

        
        # Looping through Dataframe Length, creating feather for each dataframe
        c = 0
        out = self.tensor.numpy()
        for track in out:
            np.save(path + str(c) + '.npy',track)
            c += 1
    def headerFromTensor(self):
        out = {}
        msg = self.track
        out['tempo'] = msg[9]
        out['key']   = msg[10]
        out['num']   = msg[11]
        out['den']   = msg[12]
        out['cpc']   = msg[13]
        out['nnpb']  = msg[14]
        self.header = out
    # EXPERIMENTAL
    def tensorFromFolder(self,folderName):
        path = os.getcwd() + '/' + foldername
        # For each feather in the directory, read feather into table, then append to out array
        for f in os.listdir(path):
            table = np.load(path + '/' + f)
            self.tracks.append(table)


    



In [ ]:
# Loading in track for testing
test = MidiTensor('Test16_midi.npy').track
# Pre-defining instruments
instruments = (57, 58, 59, 61, 63, 64, 66, 67, 68, 72) #57, 58, 59, 61, 63, 64, 66, 67, 68, 72
# Making Midi File
MakeMidi([test], instruments)

In [ ]:
# Test a specific track from newly made midi
mid = MidiFile('our_song_16.mid')
for i, track in enumerate(mid.tracks):
  if i != 2:
    continue
  print('Track {}: {}'.format(i, track.name))
  for msg in track:
      print(msg)

#### remake midi

In [ ]:
# Function to get the message type from the number assigned
def message_type(x):
  return {
      0: 'channel_prefix', 1: 'note_on', 2: 'sysex', 3: 'end_of_track',
        4: 'control_change', 5: 'note_off', 6: 'program_change', 
        7: 'midi_port', 8: 'pitchwheel', 9: 'sequencer_specific'
  }[x]

# Function to get the key signature from the number assigned
def getKey(x):
  return {0: 'Cb', 1: 'Gb', 2: 'Db', 3: 'Ab', 4: 'Eb', 5: 'Bb', 
                6: 'F', 7: 'C', 8: 'G', 9: 'D', 10: 'A', 11: 'E', 12: 'B', 
                13: 'F#', 14: 'C#', 15: 'Abm', 16: 'Ebm', 17: 'Bbm', 18: 'Fm', 
                19: 'Cm', 20: 'Gm', 21: 'Dm', 22: 'Am', 23: 'Em', 24: 'Bm', 
                25: 'F#m', 26: 'C#m', 27: 'G#m', 28: 'D#m', 29: 'A#m', 30: 'Cb'}[x]



# Function to do something hehe
def MakeMidi(t, *args, outname='new_song.mid'):
    # Making the midi file
    mid = MidiFile(type=1)
    trackCounter = 0
    program_counter = 0
    channel_use = [False, False, False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False]

    # If there is arugments passed in for instruments
    if (len(args[0])) > 0:
        for index, track in enumerate(t):
            # Making a track to append into the Midi File
            new_track = MidiTrack()

            # Appending the track to the midi file
            mid.tracks.append(new_track)
            first_line = True

            # Looping through each line in the track
            for line in track:

                # BUILDING HEADER TRACK
                if trackCounter == 0:
                    # Setting the basic header variables
                    # Placing tempo
                    tempo = int(abs(line[9]))
                    tempo = int(tempo/255 * 16777215)
                    if tempo > 16777215:
                        tempo = tempo % 16777215
                    if tempo == 0:
                        tempo = 500000
                    #print('Tempo:', tempo)
                    new_track.append(MetaMessage('set_tempo', tempo=tempo))

                    # Placing key Signiture
                    key = int(abs(line[10]))//10 % 30
                    if key > 30:
                        key = key % 30
                    key = getKey(int(key))
                    new_track.append(MetaMessage('key_signature', key=key, time=0))

                    # Moving onto the next track
                    trackCounter = trackCounter + 1
                    new_track = MidiTrack()
                    mid.tracks.append(new_track)
                    continue

                # Getting Message Type
                messageType = int(abs(line[0])) % 10
                #print('messageType: ', messageType)

                # If message type exists, convert form number to string
                if messageType >= 0:
                    messageType = message_type(messageType)

                # Getting the rest of the elements
                channel = int(round(abs(line[1]),0)) % 15
                note = int(round(abs(line[2]),0)) % 127
                velocity = int(round(abs(line[3]),0)) % 127
                time = int(round(abs(line[4]),0))
                control = int(round(abs(line[5]),0)) % 127
                program = int(round(abs(line[6]),0)) % 127
                value = int(round(abs(line[7]),0)) % 127
                port = int(round(abs(line[8]),0)) % 127
                tempo = int(round(abs(line[9]),0))
                key = int(round(abs(line[10]),0))
                numerator = int(round(abs(line[11]),0))
                denominator = int(round(abs(line[12]),0))
                cpc = int(round(abs(line[13]),0))
                nnpb = int(round(abs(line[14]),0))
                pitch = int(round(abs(line[15]),0)) % 30

                # Adjusting if outside of acceptable range
                if channel > 15:
                    channel = channel % 15
                if control > 127:
                    control = control % 127
                if value > 127:
                    value = value % 127
                if note > 127:
                    note = note % 127
                if velocity > 127:
                    velocity = velocity % 127
                if abs(pitch) > 8191:
                    pitch = pitch % 8191

                # ASSIGNING AN INSTRUMENT TO A TRACK
                if first_line == True:
                    # If number of instruments is less than track files, use first passed instrument for the rest
                    if program_counter < len(args[0]):
                        program = int(args[0][program_counter])
                    else:
                        program = int(args[0][0])
                    time = 0
                    # Set all channels to the same instrument
                    for channel in range(16):
                        new_track.append(Message('program_change', channel=channel,
                                         program=program, time=time))
                    program_counter += 1
                    first_line = False

                # Finding next line by message type, then appending to track
                if messageType == 'control_change':
                    channel = int(channel)
                    if channel_use[channel] == False:
                        continue
                    control = int(control)
                    value = int(value)
                    time = int(time)
                    new_track.append(Message('control_change', channel=channel,
                                     control=control, value=value, time=time))
                elif messageType == 'note_on':
                    channel = int(channel)
                    # if channel_use[channel] == True:
                    #     continue
                    note = int(note)
                    velocity = int(velocity)
                    time = int(time)
                    channel_use[channel] = True
                    new_track.append(Message('note_on', channel=channel,
                                     note=note, velocity=velocity, time=time))
                elif messageType == 'note_off':
                    channel = int(channel)
                    note = int(note)
                    velocity = int(velocity)
                    time = int(time)
                    channel_use[channel] = True
                    new_track.append(Message('note_off', channel=channel,
                                     note=note, velocity=velocity, time=time))
                elif messageType == 'pitchwheel':
                    channel = int(channel)
                    pitch = int(pitch)
                    time = int(time)
                    new_track.append(Message('pitchwheel', channel=channel, pitch=pitch, time=time))
    else:
        # Looping through all the tracks in the tensor: No Pre-defined instruments ################## Might need to update set each track with one instrument.
        for track in t:  # t.tensor
            # Making a track to append into the Midi File
            new_track = MidiTrack()

            # Appending the track to the midi file
            mid.tracks.append(new_track)

            # Looping through each line in the track
            for line in track:
                # print(line)

                # BUILDING HEADER TRACK
                if trackCounter == 0:
                    # Setting the basic header variables
                    # Placing tempo
                    tempo = int(abs(line[9]))
                    tempo = int(tempo/255 * 16777215)
                    if tempo == 0:
                        tempo = 500000
                    new_track.append(MetaMessage('set_tempo', tempo=tempo))

                    # Placing key Signiture
                    key = int(abs(line[10]))//10
                    key = getKey(int(key))
                    new_track.append(MetaMessage('key_signature', key=key, time=0))

                    # Moving onto the next track
                    trackCounter = trackCounter + 1
                    new_track = MidiTrack()
                    mid.tracks.append(new_track)
                    continue

                # Getting Message Type
                messageType = int(abs(line[0])) % 10
                #print('messageType: ', messageType)

                # If message type exists, convert form number to string
                if messageType >= 0:
                    messageType = message_type(messageType)

                # Getting the rest of the elements
                channel = int(round(abs(line[1]),0)) % 15
                note = int(round(abs(line[2]),0)) % 127
                velocity = int(round(abs(line[3]),0)) % 127
                time = int(round(abs(line[4]),0))
                control = int(round(abs(line[5]),0)) % 127
                program = int(round(abs(line[6]),0)) % 127
                value = int(round(abs(line[7]),0)) % 127
                port = int(round(abs(line[8]),0)) % 127
                tempo = int(round(abs(line[9]),0))
                key = int(round(abs(line[10]),0))
                numerator = int(round(abs(line[11]),0))
                denominator = int(round(abs(line[12]),0))
                cpc = int(round(abs(line[13]),0))
                nnpb = int(round(abs(line[14]),0))
                pitch = int(round(abs(line[15]),0)) % 30

                # Adjusting if outside of acceptable range
                if channel > 15:
                    channel = channel % 15
                if control > 127:
                    control = control % 127
                if value > 127:
                    value = value % 127
                if note > 127:
                    note = note % 127
                if velocity > 127:
                    velocity = velocity % 127
                if abs(pitch) > 8191:
                    pitch = pitch % 8191

                # Finding next line by message type, then appending to track
                if messageType == 'control_change':
                    channel = int(channel)
                    if channel_use[channel] == False:
                        continue
                    control = int(control)
                    value = int(value)
                    time = int(time)
                    new_track.append(Message('control_change', channel=channel,
                                     control=control, value=value, time=time))
                elif messageType == 'program_change':
                    channel = int(channel)
                    program = int(program)
                    time = int(time)
                    new_track.append(Message('program_change', channel=channel,
                                     program=program, time=time))
                elif messageType == 'note_on':
                    channel = int(channel)
                    note = int(note)
                    velocity = int(velocity)
                    time = int(time)
                    channel_use[channel] = True
                    new_track.append(Message('note_on', channel=channel,
                                     note=note, velocity=velocity, time=time))
                elif messageType == 'note_off':
                    channel = int(channel)
                    note = int(note)
                    velocity = int(velocity)
                    time = int(time)
                    channel_use[channel] = False
                    new_track.append(Message('note_off', channel=channel,
                                     note=note, velocity=velocity, time=time))
                elif messageType == 'pitchwheel':
                    channel = int(channel)
                    pitch = int(pitch)
                    time = int(time)
                    new_track.append(Message('pitchwheel', channel=channel, pitch=pitch, time=time))

    # Saving the newly made Midi
    mid.save(outname)


# Create NP.Arrays

In [ ]:
# used to scrape the midi files
# import requests
# base_url = "https://www.midiworld.com/download/"
# for i in range(1000,5000):
#     full_url = base_url + str(i)
#     filename = "./midis_2/" + str(i) + ".midi"
#     with open(filename, 'wb') as f:
#         f.write(requests.get(full_url).content)

In [ ]:
#!rm '/content/midis_2/3767.midi'


In [ ]:
#sums = counts = [0,0,0,0,0,0,0,0,0,0]
#j = 0
#for filename in os.listdir('trackfiles'):
#    if j > 4000: continue
#    track = MidiTensor('trackfiles/' + filename).track
#    for row in track:
#        for i in range(9):
#            if row[i] > 0:
#                counts[i] += 1
#                sums[i] += row[i]
#        if row[15] > 0: 
#            counts[9] += 1
#            sums[9] += row[15]
#    j += 1
#
#print([sums[i]/counts[i] for i in range(10)])

In [ ]:
#path = os.getcwd() + '/' + 'midis_2/'
#rate = 0
## For each feather in the directory, read feather into table, then append to out array
#for f in os.listdir(path):
#    if f.endswith('.DS_.midi'): 
#        full = path + f
#        os.remove(full)
#        continue
#    if not f.endswith('.midi'): continue
#    full = path + f
#    out = MidiTensor(full,padding = PADDING)
#    out.writeTensor(loc='averagedfiles/')
#    clear_output(wait=False)
#    os.remove(full)
#    rate += 1
#print(77/rate)

#### Pack them up

In [ ]:
#!zip -r averagedtrackfilesfull.zip averagedfiles
#clear_output(wait=False)

# VAE

In [7]:
#Actually, unpack them
# -o so we don't have to interact with this after execution. >/dev/null hides output
!unzip -o /content/trackfiles.zip >/dev/null
#clear_output(wait=False)


for filename in os.listdir('trackfiles'):
# My momma said these songs are the devil
    if filename.startswith('4196'):
        Os.remove('averagedfiles/' + filename)
    if filename.startswith('4938'):
        Os.remove('averagedfiles/' + filename)
    if filename.startswith('4939'):
        Os.remove('averagedfiles/' + filename)
# This one isn't the devil, but it's short and not very good
    if filename.startswith('4293'):
        Os.remove('averagedfiles/' + filename)

# remove erraneous files
!rm -rf /content/trackfiles/.ipynb_checkpoints

In [ ]:
#print(MidiTensor('/content/1000.midi',padding = PADDING).tracks)

In [ ]:
#def train_vae(x_train, x_test, flow=None, z_size=32, hidden_units=32, n_epochs=10):
#    def bias(y_true, y_pred):
#        import tensorflow.keras.backend as K
#        return K.mean(y_pred - y_true, axis=-1)
#    beta_update = lambda i, beta: (i+1) / n_epochs
#    vae = GatedConvVAE(PADDING, 16, flow, z_size=z_size, hidden_units=hidden_units, 
#                       output_activation='sigmoid', loss='mse', beta_update_fn=beta_update, metrics=[bias])
#    vae.fit(x_train,epochs=n_epochs, batch_size=32, validation_data=x_test)
#    return vae

In [8]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

In [ ]:
## shit to remove erraneous files 1
#path = os.getcwd() + '/trackfiles'
#
#for f in os.listdir(path):
#    if f.endswith('.npy'): continue
#    print(f)


# GlowFlow

In [9]:
def dgen():
    path = os.getcwd() + '/trackfiles'
    #path = os.getcwd() + '/averagedfiles'

    for f in os.listdir(path):
        track = MidiTensor(path + '/' + f).track
        #print(track.shape)
        track = np.array([[0.0 if np.isnan(i) else i for i in j ] for j in track] )
        #track = track[...,np.newaxis]
        #print(np.shape(track))
        yield track

In [10]:
def array_from_generator(generator, arr):
    """Fills the numpy array provided with values from
    the generator provided. Number of columns in arr
    must match the number of values yielded by the 
    generator."""
    count = 0
    for row in arr:
        try:
            item = next(generator)
        except StopIteration:
            break
        row[:] = item
        count += 1
    return arr[:count,:]

def data():
    gen = dgen()
    arr = np.empty((BATCH_SIZE,PADDING,16,1), dtype='float32')
    dataset = array_from_generator(gen, arr)
    #dataset = next(dgen())
    print(f"DATA SHAPE: {np.shape(dataset)}")
    tr_len = BATCH_SIZE * 0.8
    othr_len = BATCH_SIZE * 0.1
    return dataset[0:int(tr_len)], dataset[int(tr_len):int(tr_len+othr_len)], dataset[int(tr_len+othr_len):int(tr_len+othr_len+othr_len)] 

In [11]:
from normalizing_flows.flows import Invert
from normalizing_flows.flows.glow import GlowFlow, coupling_nn_glow
from normalizing_flows.models import FlowLVM
from normalizing_flows.models.optimization import LinearWarmupSchedule

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
n_epochs = 10
BATCH_SIZE = 40
BATCH_NUM = 0
glow = Invert(GlowFlow(num_layers=3, depth=16, coupling_nn_ctor=coupling_nn_glow(min_filters=32, max_filters=256, num_blocks=1)))
x_train, x_val, x_test = data()
#print(x_train.shape)
steps_per_epoch = x_train.shape[0] // 32
# warm-up for 10 epochs
learning_rate = LinearWarmupSchedule(2.0E-3, num_warmup_steps=20*steps_per_epoch)
glowflow = FlowLVM(glow, input_shape=(None,2000,16,1),
                optimizer=tf.keras.optimizers.Adamax(learning_rate=learning_rate))

ValueError: ignored

In [ ]:
#model.fit(x_train, x_train, epochs=1, batch_size=32, validation_data=(x_val, x_val))
from tqdm import tqdm
import normalizing_flows.utils as utils
batch_size = 32
index = 0
laps = 20
with tqdm(total=laps) as prog:
    hist = dict()
    init = tf.constant(True) # init variable for data-dependent initialization
    for i in range(laps):
        x_train, _ , _ = data()
        #print(x_train.shape)
        loss, nll, ldj  = glowflow.train_batch(x_train, init=init)
        clear_output(wait=False)
        init=tf.constant(False)
        utils.update_metrics(hist, loss=loss.numpy(), nll=nll.numpy())
        prog.update(1)
        prog.set_postfix({k: v[0] for k,v in hist.items()})
#print(model.eval_batch(x_val))
            
#model.train(x_train, steps_per_epoch=10, num_epochs=1, conditional=False)

100%|██████████| 20/20 [24:22<00:00, 73.13s/it, loss=793, nll=793]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

glowflow._init_checkpoint()
glowflow.save('checkpoint/model')
!zip -r checkpoint4.zip checkpoint/
!cp checkpoint4.zip drive/MyDrive/checkpoint4.zip

updating: checkpoint/ (stored 0%)
updating: checkpoint/model-1.data-00000-of-00001 (deflated 8%)
updating: checkpoint/checkpoint (deflated 39%)
updating: checkpoint/model-1.index (deflated 81%)
updating: checkpoint/model-2.data-00000-of-00001 (deflated 8%)
updating: checkpoint/model-2.index (deflated 81%)
updating: checkpoint/model-3.data-00000-of-00001 (deflated 8%)
updating: checkpoint/model-3.index (deflated 81%)
updating: checkpoint/model-4.index (deflated 81%)
updating: checkpoint/model-4.data-00000-of-00001 (deflated 8%)
updating: checkpoint/model-5.index (deflated 81%)
updating: checkpoint/model-5.data-00000-of-00001 (deflated 8%)
updating: checkpoint/model-6.index (deflated 81%)
updating: checkpoint/model-6.data-00000-of-00001 (deflated 8%)
  adding: checkpoint/model-7.index (deflated 81%)
  adding: checkpoint/model-7.data-00000-of-00001 (deflated 8%)


# CNN

In [12]:
def cnndata():

    gen = dgen()
    arr = np.empty((400,PADDING,16), dtype='float32')
    dataset = array_from_generator(gen, arr)
    data = []
    labels = []
    onehot = lb()
    onehot.fit(range(10))
    for track in dataset:
        td = []
        tl = []
        for x in track:
            data.append(x[1:])
            labels.append(int(x[0]))

        #labels.append(onehot.transform(tl))
    labels = onehot.transform(labels).squeeze()
    print(f"DATA SHAPE: {np.shape(data)}, LABEL SHAPE: {np.shape(labels)}")
    tr_len = BATCH_SIZE * 0.8
    othr_len = BATCH_SIZE * 0.1
    tf.clip_by_value(has_nans, -1e12, 1e12)
    return wrap(data[0:int(tr_len)]),  wrap(labels[0:int(tr_len)]),  wrap(data[int(tr_len):int(tr_len+othr_len)]),  wrap(labels[int(tr_len):int(tr_len+othr_len)]),  wrap(data[int(tr_len+othr_len):int(tr_len+othr_len+othr_len)]),  wrap(labels[int(tr_len+othr_len):int(tr_len+othr_len+othr_len)])

In [13]:
from sklearn.preprocessing import LabelBinarizer as lb
def crossEntropy(logits,label):
    pass
class featureMaker(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.featurelayer = torch.nn.Linear(in_features=15, out_features=4)
        #self.featurelayer = torch.nn.Conv1d(1,4,5)
        #self.pooling = torch.nn.Conv1d(4,1,1)
        self.softmax = torch.nn.Softmax()
        self.traininglayer = torch.nn.Linear(in_features=4, out_features=10)
        self.loss = torch.nn.CrossEntropyLoss()
        params = list(self.featurelayer.parameters()) + list(self.traininglayer.parameters())
        self.optimizer = torch.optim.Adam(params, lr=0.0001)
    def forward(self,x):
        x1  = self.featurelayer(x)
        #x2 = self.pooling(x1)
        x2  = self.traininglayer(x1)
        out = self.softmax(x2)
        print(x2)
        print(out)
        return out
        #return self.traininglayer(x1)
    def train(self,x,y,batch_size=20):
        with tqdm(total=(len(x)*2000/batch_size)) as prog:
            x = list(x.numpy())
            for j,track in enumerate(x):
                #print(track)
                preds = [] #torch.empty((10))
                labs =  []
                for i,row in enumerate(track):
                    self.optimizer.zero_grad()
                    #a, b = (i*batch_size, (i+1)*batch_size) 
                    iny = y[j][i]
                    #x, y = (data[a:b],labels[a:b])
                    #print(f"DATA SHAPE: {np.shape(inx)}, LABEL SHAPE: {np.shape(iny)}")
                    #print(row)
                    pred = self.forward(torch.tensor(row))
                    #print(pred,iny)
                    preds.append(pred)
                    labs.append(iny)
                    #output = self.loss(torch.tensor(pred), torch.tensor(iny))
                #output = self.loss(pred, iny)
                preds = torch.stack(preds)
                print(max(labs))
                labs = torch.stack(labs)
                print(preds,labs)
                output = self.loss(pred.squeeze(), labs.squeeze())
                output.backward()
                self.optimizer.step()
                prog.update(1)
                #print(output)
                prog.set_postfix({'loss':output.detach().numpy()})
    def process(self,x):
        return self.featurelayer(x)





In [ ]:
featuremaker = featureMaker()

In [ ]:

featuremaker.train(torch.tensor(xt),torch.tensor(yt))

NameError: ignored

In [117]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(15,)))
#model.add(tf.keras.layers.Dense(12, activation='linear'))
model.add(tf.keras.layers.Conv2D())
#model.add(tf.keras.layers.Dense(9, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))

#model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(10))
#model.add(tf.keras.layers.Softmax())

model.compile(optimizer=optim, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['mae'])


In [96]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(5, 3, 1)))
model.add(tf.keras.layers.Conv2D(32, 3, activation="tanh", strides=1, padding="same"))
model.add(tf.keras.layers.Conv2D(64, 2, activation="relu", strides=1, padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(109, activation="linear"))

model.add(tf.keras.layers.Dense(51, activation="linear"))
model.add(tf.keras.layers.Dense(17, activation="linear"))
model.add(tf.keras.layers.Dense(4, activation="linear"))

model.add(tf.keras.layers.Dense(10))

#model.add(tf.keras.layers.Softmax())

optim = tf.keras.optimizers.Ftrl(
    learning_rate=.01, #beta_1=.1, beta_2=.99
)
model.compile(optimizer=optim, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['mae'])


In [97]:
print(len(os.listdir('trackfiles')))


30293


In [98]:
def replacenan(t):
    return tf.where(tf.is_nan(t), tf.zeros_like(t), t)
tf.config.run_functions_eagerly(True)
onehot = lb()
onehot.fit(range(10))
i = 0
for filename in os.listdir('trackfiles'):
    if not filename.endswith('.npy'): continue
    if i < 50 or (i > 1200 and i < 1400): 
        i += 1
        continue
    print(filename)
    track = np.load('trackfiles/' + filename)
    data = []
    labels = []
    for row in track:
        if row[0]%1 != 0: continue
        row = np.nan_to_num(row, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
        new = np.reshape(row[1:],(5,3,1))
        data.append(new)
        labels.append(int(row[0]))
    labels = onehot.transform(labels).squeeze()
    # print(len(data[0]))
    # print(labels)
    model.fit(tf.constant(data),tf.constant(labels))#,batch_size=10)
    i += 1
    clear_output(wait=False)



1669_1.npy
 24/125 [====>.........................] - ETA: 3s - loss: 4.9928e-05 - mae: 9.0261

KeyboardInterrupt: ignored

In [20]:
print(model.layers)

[<keras.layers.convolutional.Conv2D object at 0x7fa2c718c790>, <keras.layers.convolutional.Conv1D object at 0x7fa2c7035d90>, <keras.layers.core.flatten.Flatten object at 0x7fa2c707e4d0>, <keras.layers.core.dense.Dense object at 0x7fa2c6f584d0>, <keras.layers.core.dense.Dense object at 0x7fa2c6f67f90>, <keras.layers.advanced_activations.Softmax object at 0x7fa2c6f6af90>]


In [101]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output = get_3rd_layer_output(tf.constant(data))
for track in layer_output:
    for row in track:
        mark = False
        for x in row:
            if x != 0.0:
                mark = True
        if mark : print(row)


[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]
[ 12.157246    1.4092572  24.809483  -29.689503 ]


In [35]:
model.save('model2')

In [182]:
print(np.load('trackfiles/4060_7.npy')[2])

[ 4.0000000e+00  8.0000000e+00  5.4208939e+01  5.7982964e+01
  0.0000000e+00  1.0000000e+00  3.2476658e+01  0.0000000e+00
  7.5899638e-02  7.9734200e+05            nan  4.0000000e+00
  4.0000000e+00  2.4000000e+01  8.0000000e+00 -2.4274861e+02]


In [123]:
model.predict(tf.constant([np.load('trackfiles/3080_2.npy')[1][1:]]))

array([[-5.1510324 ,  1.414861  ,  0.45780826, -2.1183634 , -1.0900706 ,
        -0.38665444, -2.8858557 , -4.66041   , -3.1162276 , -5.0345554 ]],
      dtype=float32)

In [ ]:
n_epochs = 10
BATCH_SIZE = 400
BATCH_NUM = 0
def bias(y_true, y_pred):
    import tensorflow.keras.backend as K
    return K.mean(y_pred - y_true, axis=-1)

beta_update = lambda i, beta: (i+1) / n_epochs
path_checkpoint = "/content/training_1/cp.ckpt"

callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=0)
#vae = GatedConvVAE(4000, 16, flow=None, z_size=32, hidden_units=32, 
vae = GatedConvVAE(PADDING,16,flow=nf.flows.affine.Planar(input_shape=tf.TensorShape([2000,16,1])), z_size=16, hidden_units=16, 
                   encoder_strides=[16,1], decoder_strides=[16,1],
                   output_activation='sigmoid', loss=nf.models.losses.wasserstein_loss(nf.models.adversarial.PatchDiscriminator((2000,16,1))), #loss='mse',
                   beta_update_fn=beta_update, metrics=[bias], callbacks=[callback])
#vae.model.summary()
#x_train, x_val, x_test = data()
for i in range(10):
    
    # PLease just try doing this as 1, I swer it will work better
    normalizationNumber = 1.0 # From testing, you need to be at least 100 to prevent loss from being > 1
    #x_train = x_train / normalizationNumber
    #x_val = x_val / normalizationNumber
    #x_test = x_test / normalizationNumber
    #BATCH_NUM += BATCH_SIZE
    vae.fit(x_train, x_train, epochs=1, batch_size=32, validation_data=(x_val, x_val)) # batch_size = 32

x_pred_vae = vae.predict(x_test)
print(x_pred_vae[3][0])


# Calculate Vectors

For this section, I set up a file structure with the top-level level folders being classification category (e.g. "Upbeat/Chill", "Bright/Soft") and subfolders being classification (e.g. "Chill", "Bright"). For the multi-subclass categories (Decade and Genre), they were in a folder with the name of the category, and then filled with the appropriate folders (50s, 60s,...,90s,10s; Rock, Pop, Jazz...). Each subclass folder was then filled with .midi files that fit that category.

In [ ]:
link = link + "/midisamples/"

# Upbeat/Chill

In [ ]:
upbeat_path = link + "UpbeatChill/Upbeat"
chill_path = link + "UpbeatChill/Chill"
upbeat_embs = embed_set(upbeat_path)
chill_embs = embed_set(chill_path)

upbeat_mean = np.mean(upbeat_embs, axis=0)
chill_mean = np.mean(chill_embs, axis=0)

UpbeatChillVector = diff_vect(upbeat_mean, chill_mean)

np.save("UCVec", UpbeatChillVector)

# Action/Exploration

In [ ]:
action_path = link + "ActionExploration/Action"
exploration_path = link + "ActionExploration/Exploration"
action_embs = embed_set(action_path)
exploration_embs = embed_set(exploration_path)

action_mean = np.mean(action_embs, axis=0)
exploration_mean = np.mean(exploration_embs, axis=0)

ActionExplorationVector = diff_vect(action_mean, exploration_mean)

np.save("AEVec", ActionExplorationVector)

# Bright/Soft

In [ ]:
bright_path = link + "BrightSoft/Bright"
soft_path = link + "BrightSoft/Soft"
bright_embs = embed_set(bright_path)
soft_embs = embed_set(soft_path)

bright_mean = np.mean(bright_embs, axis=0)
soft_mean = np.mean(soft_embs, axis=0)

BrightSoftVector = diff_vect(bright_mean, soft_mean)

np.save("BSVec", BrightSoftVector)

# Multi-Subclass Classification

In [ ]:
def max_var(means):
    max_dist = 0
    
    for i, A in enumerate(means):
        for j, B in enumerate(means):
            #print(f"({i}, {j})")
            euc_dist =  np.linalg.norm(diff_vect(A,B))
            #print(euc_dist)
            if euc_dist > max_dist:
                best_A = i
                best_B = j
                max_dist = euc_dist
    
    return best_A, best_B, max_dist

In [ ]:
path50 = link + "Decade/50s"
path60 = link + "Decade/60s"
path70 = link + "Decade/70s"
path80 = link + "Decade/80s"
path90 = link + "Decade/90s"
path10 = link + "Decade/10s"

emb50 = embed_set(path50)
emb60 = embed_set(path60)
emb70 = embed_set(path70)
emb80 = embed_set(path80)
emb90 = embed_set(path90)
emb10 = embed_set(path10)

mean50 = np.mean(emb50, axis=0)
mean60 = np.mean(emb60, axis=0)
mean70 = np.mean(emb70, axis=0)
mean80 = np.mean(emb80, axis=0)
mean90 = np.mean(emb90, axis=0)
mean10 = np.mean(emb10, axis=0)

Ai, Bi, dist = max_var([mean50,mean60, mean70, mean80, mean90, mean10])
print()
print(dist)
print(Ai)
print(Bi)

In [ ]:
FiveNineVector = diff_vect(mean50, mean90)
print(np.shape(FiveNineVector))
#print(FiveNineVector)
np.save("5090Vec", FiveNineVector)

NineOneVector = diff_vect(mean90, mean10)
print(np.shape(NineOneVector))
#print(NineOneVector)
np.save("9010Vec", NineOneVector)

In [ ]:
BritRockPath = link + "Genre/BritRock"
CountryPath = link + "Genre/Country"
CountryRockPath = link + "Genre/CountryRock"
JazzPath = link + "Genre/Jazz"
RockPath = link + "Genre/Rock"
SoftRockPath = link + "Genre/SoftRock"
SoulPath = link + "Genre/Soul"

BRemb = embed_set(BritRockPath)
Cemb = embed_set(CountryPath)
CRemb = embed_set(CountryRockPath)
Jemb = embed_set(JazzPath)
Remb = embed_set(RockPath)
SRemb = embed_set(SoftRockPath)
Semb = embed_set(SoulPath)

BRmean = np.mean(BRemb, axis=0)
Cmean = np.mean(Cemb, axis=0)
CRmean = np.mean(CRemb, axis=0)
Jmean = np.mean(Jemb, axis=0)
Rmean = np.mean(Remb, axis=0)
SRmean = np.mean(SRemb, axis=0)
Smean = np.mean(Semb, axis=0)

Ai, Bi, dist = max_var([BRmean, Cmean, CRmean, Jmean, Rmean, SRmean, Smean])
print()
print(dist)
print(Ai)
print(Bi)

## ETA 2 AM

In [ ]:
print(x_test[3][60])
print(x_pred_vae[3][60])

In [ ]:
print(vae.sample(x_test)[0][3])

In [ ]:
vae.model.summary()

In [ ]:
# Test of output from vae
# input, output = vae.sample(x_test, 1)
# input *= 100
# print(input[0][1].astype(int))
# for i in range(len(output)):
#   output[i] = output[i]*100
#   output[i] = output[i].astype(int)
# print(output)
# print(x_pred_vae[0][0])

yes = vae.model.


**Grid Search**

In [ ]:
param_grid = {
              'normalizationNumber': [1,10,20,50,100,200,256],
              'epochs':[1,5,10],
              'batch_size':[16,32]
             }


#scoring = ['loss', 'bias']
#score = make_scorer(bias, greater_is_better=False)

grid = GridSearchCV(vae, param_grid, n_jobs = -1, cv = 3, scoring='neg_log_loss') 

grid_result = grid.fit(x_train, x_train, validation_data=(x_val, x_val))

# summarize results
#print('Best:loss score is',grid_result.best_score_,'using', grid_result.best_params_)
loss = grid_result.cv_results_['loss']

params = grid_result.cv_results_['params']

for mean, stdev, param in zip(loss, params):
    print("%f (%f) with: %r" % (loss, param))


In [ ]:
print(x_train)

In [ ]:
np.save('final.npy',x_pred_vae)

In [ ]:
MakeMidi(x_pred_vae)